# <center> Данный скрипт осуществляет выгрузку данных с комментариями и пользователями,  для их дальнейшего анализа

In [1]:
# Для выгрузки данных с ВК
import requests
import json
import os
from auth_data import token # сервисный токен
from auth_data import tokenbgd # личный токен

# Для обработки полученных данных
import pandas as pd
import numpy as np

# <center> Получаем данные постов группы

In [116]:
# Запрашиваемые параметры
group_name = [# Название группы взятое из ссылки
    'atheism',
    'varlamov',
    'sandy_mustache_vk',
    'rocknrofl',
    'agnpage', 
    'pinklamborghini',
    'irkdtp' 
]
count_last_posts = 40 # Число последних постов

# Создаем список с id номерами постов группы
group_json = list()
post_id_dict = dict()

for j in range(len(group_name)):
    # Производим запрос
    url_group = f"https://api.vk.com/method/wall.get?domain={group_name[j]}&count={count_last_posts}&access_token={token}&v=5.81"
    req = requests.get(url_group)
    group_json.append(req.json())
    post_id_list = list()
    for i in range(len(group_json[j]['response']['items'])):
        post_id_list.append(group_json[j]['response']['items'][i]['id'])
        # Проверяем существует ли директория с именем группы
    if os.path.exists(f"groups/{group_name[j]}"):
        print(f"Директория с именем {group_name[j]} уже существует!")
    else:
        os.makedirs(f"groups/{group_name[j]}")
        # сохраняем данные в json файл, чтобы видеть структуру
    post_id_dict.update({group_name[j]:post_id_list})
        
    with open(f"groups/{group_name[j]}/{group_name[j]}.json", "w", encoding="utf-8") as file:
        json.dump(group_json, file, indent=4, ensure_ascii=False)

print("id постов:",post_id_dict)

Директория с именем atheism уже существует!
Директория с именем varlamov уже существует!
Директория с именем sandy_mustache_vk уже существует!
Директория с именем rocknrofl уже существует!
Директория с именем agnpage уже существует!
Директория с именем pinklamborghini уже существует!
Директория с именем irkdtp уже существует!
id постов: {'atheism': [2797570, 2797520, 2797453, 2797383, 2796891, 2796777, 2796572, 2796375, 2796085, 2795927, 2795851, 2795799, 2795705, 2795657, 2795524, 2795466, 2795430, 2795162, 2794790, 2794407, 2794126, 2793803, 2793651, 2793616, 2793544, 2793477, 2793427, 2793321, 2793191, 2793003, 2792911, 2792632, 2792491, 2792311, 2792101, 2792039, 2791899, 2791772, 2791621, 2791384], 'varlamov': [2364754, 2365696, 2365605, 2365553, 2365377, 2365274, 2365218, 2365039, 2365022, 2365006, 2364855, 2364792, 2364788, 2364719, 2364654, 2364500, 2363995, 2363976, 2363946, 2363885, 2363863, 2363826, 2363778, 2363738, 2363697, 2363675, 2363660, 2363587, 2363537, 2363456, 2363

# <center> Получение комментариев

In [117]:
# Создаем списки с пользователями и текстом комментариев
groups_id = list()
text = list()
post_id_json = list()

for i in range(len(group_name)):
    groups_id.append(group_json[i]['response']['items'][0]['from_id'])

for m in range(len(groups_id)):
    for j in range(len(post_id_dict[group_name[m]])):
        # Производим запрос
        url_posts = f"https://api.vk.com/method/wall.getComments?owner_id={groups_id[m]}&post_id={post_id_dict[group_name[m]][j]}&count=100&access_token={token}&v=5.81"
        req = requests.get(url_posts)
        post_id_json.append(req.json())
        for i in range(len(post_id_json[j]['response']['items'])):
            text.append([
                groups_id[m],                                               # id группы
                post_id_dict[group_name[m]][j],                                        # id поста
                post_id_json[j]['response']['items'][i]['from_id'],     # id пользователя 
                post_id_json[j]['response']['items'][i]['text']],       # текст комментария            
            )

## Проверяем существует ли директория с именем группы
#if os.path.exists(f"groups/{group_name}/{groups_id}/{groups_id}_text")==False:
#    os.makedirs(f"groups/{group_name}/{groups_id}/{groups_id}_text")

## Сохраняем данные в json файл, чтобы видеть структуру       
#with open(f"groups/{group_name}/{groups_id}/{groups_id}_text/{groups_id}_text.json", "w", encoding="utf-8") as file:
#    json.dump(text, file, indent=4, ensure_ascii=False)


# <center> Создаем таблицу, куда будем вносить данные о комментариях и пользователях

In [133]:
users_data = pd.DataFrame(
    data = text,
    columns= ['id_группы', 'id_поста', 'id_пользователя', 'Комментарий']
)

In [134]:
users_data.head()

,id_группы,id_поста,id_пользователя,Комментарий
0,-63683472,2797570,4819218,"Галкин против власти, но не против баблишка"
1,-63683472,2797570,583369270,
2,-63683472,2797570,6348835,"хотел подьебать власть,а подьебал себя"
3,-63683472,2797570,178652403,Первый раз с ним согласен
4,-63683472,2797570,182375107,"[id4819218|Виталий], ну это как наши чиновники..."


In [135]:
print(f"Получены данные о {users_data.shape[0]} комментариях!")

Получены данные о 21406 комментариях!


In [136]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21406 entries, 0 to 21405
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_группы        21406 non-null  int64 
 1   id_поста         21406 non-null  int64 
 2   id_пользователя  21406 non-null  int64 
 3   Комментарий      21406 non-null  object
dtypes: int64(3), object(1)
memory usage: 669.1+ KB


# <center> Добавление данных об имени, фамилии, дате рождения, закрытости и городе пользователя

In [137]:
# Напишем функцию, возвращающую имя пользователя
def user_info(user_id, token=token): 
    # Производим запрос
    url_posts = f"https://api.vk.com/method/users.get?user_ids={user_id}&fields=city,bdate,is_closed&access_token={token}&v=5.81"
    req = requests.get(url_posts)
    users_info_json = req.json()    

    try:
        first_name  = users_info_json['response'][0]['first_name']
        last_name   = users_info_json['response'][0]['last_name']
        try:
            birthday    = users_info_json['response'][0]['bdate']
        except KeyError:
            birthday = 0
        try:
            zakritost   = users_info_json['response'][0]['is_closed']
        except KeyError:
            zakritost = 0
        try:
            city        = users_info_json['response'][0]['city']['title']         
        except KeyError:
            city = 0  
    #except UnboundLocalError:
    #    return [first_name, last_name, '', city]
    except IndexError:
        return 0, 0, 0, 0, 0

    return [first_name, last_name, birthday, zakritost, city]

In [138]:
user_info_df = pd.DataFrame()
user_info_df['id_пользователя'] = users_data['id_пользователя'].unique()
user_info_df['Инфо'] = user_info_df['id_пользователя'].apply(user_info)

user_info_df['Имя'] = user_info_df['Инфо'].apply(lambda x: x[0])
user_info_df['Фамилия'] = user_info_df['Инфо'].apply(lambda x: x[1])
user_info_df['Дата_рождения'] = user_info_df['Инфо'].apply(lambda x: x[2])
user_info_df['Закрытость профиля'] = user_info_df['Инфо'].apply(lambda x: x[3])
user_info_df['Город'] = user_info_df['Инфо'].apply(lambda x: x[4])

user_info_df = user_info_df.drop(['Инфо'], axis=1) 

user_info_df.head(2)

,id_пользователя,Имя,Фамилия,Дата_рождения,Закрытость профиля,Город
0,4819218,Vitaly,Vasilchenko,13.1,0,Rostov-on-Don
1,583369270,Martin,Ulrikh,0,0,0


#### <center> Берем данные о пользователях из "**user_info_df**" и дополняем ими таблицу с данными о комментариях "**users_data**".

In [139]:
print(f"Получены данные о {user_info_df.shape[0]} пользователях!")

Получены данные о 830 пользователях!


In [140]:
users_data = users_data.merge(
    user_info_df,
    on='id_пользователя',
    how='left'
)
users_data.head(3)

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Дата_рождения,Закрытость профиля,Город
0,-63683472,2797570,4819218,"Галкин против власти, но не против баблишка",Vitaly,Vasilchenko,13.1,0,Rostov-on-Don
1,-63683472,2797570,583369270,,Martin,Ulrikh,0,0,0
2,-63683472,2797570,6348835,"хотел подьебать власть,а подьебал себя",Roma,Cherny,1.6,0,0


In [ ]:
https://vk.com/irkdtp?w=wall-37432351_2951579

In [154]:
group_name

['atheism',
 'varlamov',
 'sandy_mustache_vk',
 'rocknrofl',
 'agnpage',
 'pinklamborghini',
 'irkdtp']

In [155]:
groups_id

[-63683472, -24946565, -76677520, -169538908, -93574769, -182169856, -37432351]

In [151]:
# Генерируем ссылку на пост с комментариями
def url_to_post(id_group, id_post):
    return f"https://vk.com/{id_group}?w=wall{id_group}_{id_post}"

users_data['Ссылка_на_пост'] = 0
for i in range(users_data.shape[0]):
    id_group = users_data['id_группы'].iloc[i]
    id_post = users_data['id_поста'].iloc[i]
    users_data['Ссылка_на_пост'].iloc[i] = url_to_post(id_group, id_post)

c:\Users\mish9\.conda\envs\sf\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [142]:
# Генерируем ссылку на аккаунт пользователя
def url_to_user(id_user):
    return f"https://vk.com/id{id_user}"

users_data['Ссылка_на_пользователя'] = users_data['id_пользователя'].apply(url_to_user)

In [152]:
users_data.head()

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Дата_рождения,Закрытость профиля,Город,Ссылка_на_пост,Ссылка_на_пользователя
0,-63683472,2797570,4819218,"Галкин против власти, но не против баблишка",Vitaly,Vasilchenko,13.1,0,Rostov-on-Don,https://vk.com/-63683472?w=wall-63683472_2797570,https://vk.com/id4819218
1,-63683472,2797570,583369270,,Martin,Ulrikh,0,0,0,https://vk.com/-63683472?w=wall-63683472_2797570,https://vk.com/id583369270
2,-63683472,2797570,6348835,"хотел подьебать власть,а подьебал себя",Roma,Cherny,1.6,0,0,https://vk.com/-63683472?w=wall-63683472_2797570,https://vk.com/id6348835
3,-63683472,2797570,178652403,Первый раз с ним согласен,Kirill,Ermolov,4.5.1984,0,Balashikha,https://vk.com/-63683472?w=wall-63683472_2797570,https://vk.com/id178652403
4,-63683472,2797570,182375107,"[id4819218|Виталий], ну это как наши чиновники...",Kirochka,Dmitrieva,22.9,0,Chelyabinsk,https://vk.com/-63683472?w=wall-63683472_2797570,https://vk.com/id182375107


# <center> Отфильтровываем данные по пользователям из Иркутской области

In [145]:
from goroda import Irk_obl_list

C:\Users\mish9\AppData\Local\Temp/ipykernel_14384/1471147315.py:1: DtypeWarning: Columns (9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  from goroda import Irk_obl_list


In [153]:
# Отфильтровываем только пользователей из списка интересующих городов
Irkutsk = users_data[users_data['Город'].apply(lambda x: x in Irk_obl_list)]
Irkutsk

,id_группы,id_поста,id_пользователя,Комментарий,Имя,Фамилия,Дата_рождения,Закрытость профиля,Город,Ссылка_на_пост,Ссылка_на_пользователя
108,-63683472,2797520,69546585,🤣,Kristina,Kopytova,2.8,0,Rybinsk,https://vk.com/-63683472?w=wall-63683472_2797520,https://vk.com/id69546585
222,-63683472,2796891,29533610,"[id40370025|Александр], кто про что а кремлебо...",Alexander,Bolshunov,25.2,0,Kirov,https://vk.com/-63683472?w=wall-63683472_2796891,https://vk.com/id29533610
269,-63683472,2796891,305433238,"[id231159725|Ольга], а может лучше пускай никт...",Rewverz,Rewverz,13.7.1988,0,Samara,https://vk.com/-63683472?w=wall-63683472_2796891,https://vk.com/id305433238
277,-63683472,2796891,305433238,"[id231159725|Ольга], 2022 Российское вторжение...",Rewverz,Rewverz,13.7.1988,0,Samara,https://vk.com/-63683472?w=wall-63683472_2796891,https://vk.com/id305433238
290,-63683472,2796891,305433238,"[id231159725|Ольга], а если бы Германия первой...",Rewverz,Rewverz,13.7.1988,0,Samara,https://vk.com/-63683472?w=wall-63683472_2796891,https://vk.com/id305433238
...,...,...,...,...,...,...,...,...,...,...,...
20286,-37432351,2952160,57574596,"[id426871262|Омич], вранье",Onatoly,Zotov,28.4,0,Samara,https://vk.com/-37432351?w=wall-37432351_2952160,https://vk.com/id57574596
20414,-37432351,2951968,266884711,"[id322498964|Sergey], российский народ намного...",Pavel,Khadanov,3.1.1986,0,Ulan-Ude,https://vk.com/-37432351?w=wall-37432351_2951968,https://vk.com/id266884711
21113,-37432351,2951617,757271800,"Это он замеряет диаметр очка ,в которое летит ...",German,Ivanov,8.9.1985,0,Samara,https://vk.com/-37432351?w=wall-37432351_2951617,https://vk.com/id757271800
21208,-37432351,2951579,87736548,Военная авантюра нашего фюрера путина и его др...,Eduard,Trubacheev,0,0,Ulan-Ude,https://vk.com/-37432351?w=wall-37432351_2951579,https://vk.com/id87736548


In [150]:
Irkutsk.loc[21208]['Комментарий']

'Военная авантюра нашего фюрера путина и его дружков олигархов терпит полный крах на Украине... Все  разговоры, что мы кого- то защищаем, гроша ломаного не стоят... Люди начинают осознавать ложь, про НАТО( сейчас  мы не воюем  с НАТО , но с января НАТО официально начнут помогать Украине, авиацией и ракетами), нацистов(у нас в России нациков в разы больше чем в Украине) и биолаборатории(пока ни одной ещё не нашли)... Мы не право Социализма несём на Украину... Не Советскую власть восстанавливаем...Это спор разных хозяйствующих буржуазных субъектов за деньги, власть, влияние...\nОн положит всех спасая свой полированный зад...'

In [96]:
#users_data.to_csv('users_data.csv', sep=';', decimal='.', index=False)